# Import Libs

In [1]:
import pandas as pd
import numpy as np
from statistics import mean

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score, confusion_matrix


import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import InputLayer, Dense, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling2D, Flatten, Embedding, Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

# Load dataset

In [2]:
df = pd.read_csv('./dataset/hsd_pre_processed.csv')

In [3]:
df

,Unnamed: 0,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text
0,0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...
1,1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...
2,2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras
3,3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...
4,4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha
...,...,...,...,...,...,...,...,...,...,...
5665,5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...
5666,5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra
5667,5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher
5668,5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...


# Feature Selection

## CHI-2

In [4]:
X = df['pre_processed_text']
y = df['hatespeech_comb']


def feature_selection_chi2(X,y):
  normalizer = MinMaxScaler()
  X_norm = normalizer.fit_transform(X)
  chi_selector = SelectKBest(chi2, k=241)
  chi_selector.fit(X_norm, y)

  chi_support = chi_selector.get_support()
  selected_features = np.where(chi_support)[0]
  #chi_feature = X.loc[:,chi_support].columns.tolist()
  #print(str(len(selected_features)), 'selected features')
  return selected_features


## Bag of Words

In [5]:
def bag_of_words(X_train, X_test, n_grams):
    vectorizer = CountVectorizer(ngram_range=(1, n_grams))
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    return X_train,X_test

# Split into training and test sets

In [6]:
X = df['pre_processed_text']
y = df['hatespeech_comb']

RANDOM_STATE = 42

# Hold out
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, train_size = 0.8, random_state = RANDOM_STATE)
for i, (train_index, test_index) in enumerate(sss.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Training

## CNN

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, f1_score, accuracy_score
from tensorflow.keras.optimizers import RMSprop
from statistics import mean

In [17]:
activation_func = 'relu'
epochs = 10
batch_size = 128
learning_rate = 0.001
max_len = 50000
vocab_size = 20000  # Número total de palavras no vocabulário
embedding_dim = 100  # Dimensão dos embeddings
filters = 100  # Número de filtros para a camada Conv1D
kernel_size = 3  # Tamanho do kernel na convolução

def cnn():
    model = Sequential()
    model.add(InputLayer(shape=(max_len,)))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(Conv1D(10, kernel_size=kernel_size, activation=activation_func))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(20, kernel_size=kernel_size, activation=activation_func))
    model.add(Flatten())
    model.add(Dense(units=100, activation=activation_func))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    
    adam = Adam(learning_rate=learning_rate, clipvalue=0.5)
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['binary_accuracy'])
    model.summary()
    return model

In [18]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8, random_state=42)

results = []
f1 = []
accuracy = []

X = df['pre_processed_text']
y = df['hatespeech_comb']
n_gram = 2

print("# Training")

for i, (train_index_cv, val_index) in enumerate(sss.split(X_train, y_train)):
    print(f"Folder :{i}")
    X_train_cv, X_val = X[train_index_cv], X[val_index]
    y_train_cv, y_val = y[train_index_cv], y[val_index]

    # bag of words
    X_train_cv, X_val = bag_of_words(X_train_cv, X_val, n_gram)

    # Padding
    X_train_cv = pad_sequences(X_train_cv, maxlen=max_len, padding='post')
    X_val = pad_sequences(X_val, maxlen=max_len, padding='post')


    # Model
    model = KerasClassifier(model=cnn,
                        epochs=epochs,
                        batch_size=batch_size)
    
    # Fit
    model.fit(X_train_cv, y_train_cv)
    pred = model.predict(X_val)

    result = classification_report(y_val, pred)
    results.append(result)

    f = f1_score(y_val, pred)
    f1.append(f)
    print(f"# F1: {f}")

    acc = accuracy_score(y_val, pred)
    accuracy.append(acc)
    print(f"# Accuracy: {acc}")
    print("===============")

print("# Mean Accuracy: ", mean(accuracy))
print("# Mean F1: ", mean(f1))

# Training
Folder :0


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 50000, 100)     │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 49998, 10)      │         3,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 24999, 10)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 24997, 20)      │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 499940)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 100)            │    49,994,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,997,831 (198.36 MB)

 Trainable params: 51,997,831 (198.36 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 50000, 100)     │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 49998, 10)      │         3,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 24999, 10)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 24997, 20)      │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 499940)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 100)            │    49,994,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,997,831 (198.36 MB)

 Trainable params: 51,997,831 (198.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - binary_accuracy: 0.6343 - loss: 27.3116
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - binary_accuracy: 0.7077 - loss: 0.5787
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - binary_accuracy: 0.8375 - loss: 0.3977
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - binary_accuracy: 0.8516 - loss: 0.6950
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - binary_accuracy: 0.8923 - loss: 0.2806
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - binary_accuracy: 0.9777 - loss: 0.0899
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - binary_accuracy: 0.9910 - loss: 0.0385
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - binary_accuracy: 0.9945 - loss: 0.0205
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - binary_accuracy: 0.9953 - loss: 0.0149
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - binary_accuracy: 0.9952 - loss: 0.0145
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 255ms/step
# F1: 0.4826086956521739
# Accuracy: 0.7378854625

# Evaluation

In [19]:
model = KerasClassifier(model = cnn,
                        epochs = epochs,
                        batch_size = batch_size)


X_train, X_test = bag_of_words(X_train, X_test, n_gram)


# Padding
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

model.fit(X_train, y_train)
pred = model.predict(X_test)

result = classification_report(y_test, pred)
f1 = f1_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)

print(result)

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ (None, 50000, 100)     │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 49998, 10)      │         3,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 24999, 10)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 24997, 20)      │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 499940)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 100)            │    49,994,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,997,831 (198.36 MB)

 Trainable params: 51,997,831 (198.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - binary_accuracy: 0.6435 - loss: 14.9091
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.6822 - loss: 0.6560
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.6670 - loss: 4.9949
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.7486 - loss: 0.5199
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.9040 - loss: 0.2538
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.9716 - loss: 0.1136
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.9878 - loss: 0.0506
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.9892 - loss: 0.0365
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - binary_accuracy: 0.9940 - loss: 0.0191
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - binary_accuracy: 0.9958 - loss: 0.0157
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step
              precision    recall  f1-score   sup